In [19]:
import pandas as pd

# Read the first excel sheet into a pandas DataFrame
df1 = pd.read_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/HK-Races.xlsx')

# Read the second excel sheet into a pandas DataFrame
df2 = pd.read_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/HK-Runs.xlsx')


# Set the ID column as the index for both DataFrames
df1.set_index('race_id', inplace=True)
df2.set_index('race_id', inplace=True)

# Get the column(s) from df1 to add to df2
columns_to_add = ['date','venue','race_no','config','surface','distance','going',
                  'horse_ratings','prize','race_class','sec_time1','sec_time2','sec_time3',
                  'sec_time4','sec_time5','sec_time6','sec_time7','time1','time2','time3',
                  'time4','time5','time6','time7','place_combination1','place_combination2',
                  'place_combination3','place_combination4','place_dividend1','place_dividend2',
                  'place_dividend3','place_dividend4','win_combination1','win_dividend1',
                  'win_combination2','win_dividend2']
# Add the columns to df2 using the ID column to match rows
df2[columns_to_add] = df1[columns_to_add]

# Save the updated DataFrame to a new excel file
df2.to_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/HorseDataset.xlsx')



In [149]:
import pandas as pd

# Read the Excel file into a pandas DataFrame
# df = pd.read_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/Testing-File.xlsx')
df = pd.read_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/HorseDataset.xlsx')


# Calculate the average actual weight for each race id
avg_weights = df.groupby('race_id')['actual_weight'].transform('mean')

# Add the new column to the DataFrame
df['Mean_Average_by_RaceID'] = avg_weights

# Calculate the difference between actual weight and average weight for each row
diff = df['actual_weight'] - df['Mean_Average_by_RaceID']

# Add the difference column to the DataFrame
df['Mean_Difference'] = diff

# Convert the date column to a datetime format
df['date'] = pd.to_datetime(df['date'])

# Find the day of the week for each row
df['day_of_week'] = df['date'].dt.day_name()

df['is_weekday'] = df['date'].dt.weekday.apply(lambda x: 0 if x < 5 else 1)


# Sort the DataFrame by horse_id and date in descending order
df = df.sort_values(by=['horse_id', 'date'], ascending=[True, False])

# Create a new Performance column with the shifted result values
df['Performance'] = df.groupby('horse_id')['result'].shift(-1)

# Fill "missing" for the last row of each race_id in the Performance column
df['Performance'] = df.groupby('race_id')['Performance'].fillna('missing')

# Sort the DataFrame by horse_id and date in descending order
df = df.sort_values(by=['race_id'], ascending=[True])

# Find the minimum win_odds for each race_id
min_win_odds = df.groupby('race_id')['win_odds'].min()

# Create a new min_win_odd column and populate it with the minimum win_odds for each race_id
df['min_win_odd'] = df['race_id'].map(min_win_odds)

# Create a new favorite_odd column and populate it based on the difference between min_win_odd and win_odds
df['favorite_odd'] = (df['min_win_odd'] - df['win_odds'] == 0).astype(int)

# Fill the blank prize values with the string 'missing'
df['prize'].loc[df['prize'].isnull() | df['prize'].empty] = 'missing'

# Create the Blinkers column and set its values based on horse_gear
df['BlinkersB'] = 0  # Initialize all values to 0
df.loc[df['horse_gear'].str.contains('B'), 'BlinkersB'] = 1
df.loc[df['horse_gear'].str.contains('BB'), 'BlinkersB'] = 1
df.loc[df['horse_gear'] == 'B', 'BlinkersB'] = 1 
df.loc[df['horse_gear'] == '--', 'BlinkersB'] = 0
df.loc[(df['BlinkersB'] != 1), 'BlinkersB'] = 0  # Set all other values to 0

df['BlinkersT'] = 0  # Initialize all values to 0
df.loc[df['horse_gear'].str.contains('T'), 'BlinkersT'] = 1
df.loc[df['horse_gear'].str.contains('TT'), 'BlinkersT'] = 1
df.loc[df['horse_gear'] == 'T', 'BlinkersT'] = 1
df.loc[df['horse_gear'] == '--', 'BlinkersT'] = 0
df.loc[df['horse_gear'] == 'V', 'BlinkersT'] = 0
df.loc[(df['BlinkersT'] != 1), 'BlinkersT'] = 0  # Set all other values to 0

df['BlinkersV'] = 0  # Initialize all values to 0
df.loc[df['horse_gear'].str.contains('V'), 'BlinkersV'] = 1
df.loc[df['horse_gear'] == 'V', 'BlinkersV'] = 1 
df.loc[df['horse_gear'].str.contains('B'), 'BlinkersV'] = 0
df.loc[df['horse_gear'].str.contains('BB'), 'BlinkersV'] = 0
df.loc[df['horse_gear'] == 'T', 'BlinkersV'] = 0
df.loc[df['horse_gear'] == 'B', 'BlinkersV'] = 0
df.loc[df['horse_gear'] == '--', 'BlinkersV'] = 0
df.loc[(df['BlinkersV'] != 1), 'BlinkersV'] = 0  # Set all other values to 0
       
# NEW COLUMN HORSE_SEX : IF THE HORSE TYPE IS BROWN, GREY, ROAN, BLANKS --> MARK THESE ROWS AS MISSING
# OTHERWISE I WILL KEEP OTHER VALUES
# NEW COLUMN HORSE_SEX_VALUE : 
#     MISSING - MISSING
#     Gelding - 0
#     Mare, Filly - 1
#     Colt , Horse, RIG -2

# Create a new column called "New_Horse_Sex"
df['New_Horse_Sex'] = df['horse_type']

# Define the list of values for which the new column should be populated as "missing"
missing_values = ['brown', 'grey', 'roan', '']
Gelding_values = ['gelding']
Mare_values = ['mare', 'filly']
Colt_values = ['colt' , 'horse', 'rig']

# Convert the values in the "HORSE_SEX" column and the missing values to lowercase or uppercase
df['horse_type'] = df['horse_type'].str.lower()
missing_values = [x.lower() for x in missing_values]
Gelding_values = [x.lower() for x in Gelding_values]
Mare_values = [x.lower() for x in Mare_values]
Colt_values = [x.lower() for x in Colt_values]


# Loop through the rows of the dataframe and update the values of the new column
for index, row in df.iterrows():
    if row['horse_type'] in missing_values:
        df.at[index, 'New_Horse_Sex'] = 'missing'
    
# Apply the lambda function to the "horse_sex" column and create a new column called "horse_sex_value"
df['horse_sex_value'] = df['New_Horse_Sex'].apply(lambda x: 'missing' 
                                              if pd.isna(x) or str(x).lower() in ['', 'brown', 'grey', 'roan'] 
                                              else 0 if str(x).lower() == 'gelding' 
                                              else 1 if str(x).lower() in ['mare', 'filly'] 
                                              else 2 if str(x).lower() in ['colt', 'horse', 'rig'] 
                                              else 'missing' if str(x).lower() in ['missing']
                                              else None)

# ADD COLUMN : DAYS DIFFERENCE
# SORT BY HORSE ID ASC
# SORT BY DATE DESC
# FIND THE DIFFERENCE OF PRIOR RACE AND CAPTURE... LAST ROW OF EACH RACE COULD BE MISSING
# Sort the dataframe by horse_id in ascending order and date in descending order
df = df.sort_values(['horse_id', 'date'], ascending=[True, False])

# Create a new column called DAYS_DIFFERENCE
df['days_difference'] = ''

# Loop through each horse_id and calculate the difference in days between consecutive rows
for id in df['horse_id'].unique():
    rows = df[df['horse_id'] == id].index
    for i in range(len(rows)-1):
        first_row = rows[i]
        second_row = rows[i+1]
        third_row = rows[i+2] if i+2 < len(rows) else None
        
        days_diff1 = (df.loc[first_row, 'date'] - df.loc[second_row, 'date']).days
        df.loc[first_row, 'days_difference'] = days_diff1
        
        if third_row is not None:
            days_diff2 = (df.loc[second_row, 'date'] - df.loc[third_row, 'date']).days
            df.loc[second_row, 'days_difference'] = days_diff2
            
    last_row = rows[-1]
    df.loc[last_row, 'days_difference'] = 'missing'


# Sort the DataFrame by horse_id and date in descending order
df = df.sort_values(by=['race_id'], ascending=[True])

# ADD COLUMN JOCKEY CHANGE

# SORT BY HORSE ID ASC
# SORT BY DATE DESC
#COMPARE THE COMPARE 34 AND 63 --> FIND THE DIFFERENCE. 1
## SAME VALUES WILL BE 0
## LAST ROW WILL BE 1


# Sort by horse_id ascending and date descending
df = df.sort_values(by=['horse_id', 'date'], ascending=[True, False])

# Create the jockey_change column
df['jockey_change'] = 0

# Iterate through each horse_id group
for name, group in df.groupby('horse_id'):
    # Get the jockey_id values for the group
    jockey_ids = group['jockey_id'].values
    # Set the last row jockey_change to 1
    df.loc[group.index[-1], 'jockey_change'] = 1
    # Iterate through the rows in the group, comparing consecutive jockey_id values
    for i in range(len(jockey_ids) - 1):
        if jockey_ids[i] == jockey_ids[i+1]:
            df.loc[group.index[i], 'jockey_change'] = 0
        else:
            df.loc[group.index[i], 'jockey_change'] = 1

# Sort the DataFrame by horse_id and date in descending order
df = df.sort_values(by=['race_id'], ascending=[True])
            
# ADD COLUMN TRAINER  CHANGE
# SORT BY HORSE ID ASC
# SORT BY DATE DESC
#COMPARE THE COMPARE ROW 2 AND 3 --> FIND THE DIFFERENCE AND IF THERE IS A CHANGE, IT IS 1
## SAME VALUES WILL BE 0
## LAST ROW WILL BE 1


# Sort by horse_id ascending and date descending
df = df.sort_values(by=['horse_id', 'date'], ascending=[True, False])

# Create the jockey_change column
df['trainer_change'] = 0

# Iterate through each horse_id group
for name, group in df.groupby('horse_id'):
    # Get the jockey_id values for the group
    jockey_ids = group['trainer_id'].values
    # Set the last row jockey_change to 1
    df.loc[group.index[-1], 'trainer_change'] = 1
    # Iterate through the rows in the group, comparing consecutive jockey_id values
    for i in range(len(jockey_ids) - 1):
        if jockey_ids[i] == jockey_ids[i+1]:
            df.loc[group.index[i], 'trainer_change'] = 0
        else:
            df.loc[group.index[i], 'trainer_change'] = 1

# Sort the DataFrame by horse_id and date in descending order
df = df.sort_values(by=['race_id'], ascending=[True])

# Save the updated DataFrame to a new Excel file
# df.to_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/Testing-File-2.xlsx', index=False)
df.to_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/HorseDataset-2.xlsx', index=False)


# Read the Excel file into a pandas DataFrame
df1 = pd.read_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/HorseDataset-2.xlsx')
# df.to_excel(f'/Users/kalaivani/Documents/Spring 2023/Stad model/Project/Final/Testing-File-2.xlsx', index=False)

# get list of column names
column_names = df.columns

# print column names
print(column_names)


/Users/kalaivani/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Index(['race_id', 'horse_no', 'horse_id', 'result', 'won', 'lengths_behind',
       'horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'Post Pos',
       'position_sec1', 'position_sec2', 'position_sec3', 'position_sec4',
       'position_sec5', 'position_sec6', 'behind_sec1', 'behind_sec2',
       'behind_sec3', 'behind_sec4', 'behind_sec5', 'behind_sec6', 'time1',
       'time2', 'time3', 'time4', 'time5', 'time6', 'finish_time', 'win_odds',
       'place_odds', 'trainer_id', 'jockey_id', 'date', 'venue', 'race_no',
       'config', 'surface', 'distance', 'going', 'horse_ratings', 'prize',
       'race_class', 'sec_time1', 'sec_time2', 'sec_time3', 'sec_time4',
       'sec_time5', 'sec_time6', 'sec_time7', 'time7', 'place_combination1',
       'place_combination2', 'place_combination3', 'place_combination4',
       'place_dividend1', 'place_dividend2', 'place_dividend3',
       'place_dividend4', 'win_combination1', 'w